In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

"Data Import and Formatting"

# Open json as DataFrame
dataset = "WoodbineAll.json"
data = pd.read_json(dataset)

# Convert CollectionDate into datetime object 
datetimes = pd.to_datetime(data['CollectionDate'])

# Add date-time to new columns
data['year'] = datetimes.dt.year
data['month'] = datetimes.dt.month
data['day'] = datetimes.dt.day

# Extract information from the data column
beach_data = pd.DataFrame([elems[0] for elems in data['data'] if elems is not None])

# And concatenate to the original dataframe 
data = pd.concat([data, beach_data], axis=1)

# Drop redundant columns
data = data.drop(['CollectionDate', "data"], axis=1)

# Drop weird column
data = data.drop("statusFlagPre", axis=1) # We don't know what this describes 

# print(data['advisory'].unique())

"Data Cleaning"

data = data.dropna(subset="statusFlag")

print(data.info())

# Print ecoli NaN entries
print(data.drop(['beachName', 'beachId'], axis=1).loc[data['eColi'].isna()]) # Some entries have the "safe" status despite no data

# Print unique status flags
print(data['statusFlag'].unique()) # Consider dropping NO_DATA and UNTESTED entries?

<class 'pandas.core.frame.DataFrame'>
Index: 1673 entries, 0 to 1672
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1673 non-null   int32  
 1   month       1673 non-null   int32  
 2   day         1673 non-null   int32  
 3   beachId     1673 non-null   float64
 4   beachName   1673 non-null   object 
 5   eColi       1657 non-null   float64
 6   advisory    1673 non-null   object 
 7   statusFlag  1673 non-null   object 
dtypes: float64(2), int32(3), object(3)
memory usage: 98.0+ KB
None
      year  month  day  eColi  \
80    2024      6   20    NaN   
81    2024      6   19    NaN   
440   2020      8   30    NaN   
441   2020      8   29    NaN   
465   2020      8    5    NaN   
467   2020      8    3    NaN   
697   2018      6   17    NaN   
838   2016      8   26    NaN   
839   2016      8   25    NaN   
1176  2013      8    7    NaN   
1177  2013      8    6    NaN   
1523  2010      7    9   

In [4]:
data.head(15)

,year,month,day,beachId,beachName,eColi,advisory,statusFlag
0,2024,9,8,8.0,Woodbine Beaches,16.0,E.coli levels are within the City of Toronto's...,SAFE
1,2024,9,7,8.0,Woodbine Beaches,19.0,E.coli levels are within the City of Toronto's...,SAFE
2,2024,9,6,8.0,Woodbine Beaches,19.0,E.coli levels are within the City of Toronto's...,SAFE
3,2024,9,5,8.0,Woodbine Beaches,17.0,E.coli levels are within the City of Toronto's...,SAFE
4,2024,9,4,8.0,Woodbine Beaches,20.0,E.coli levels are within the City of Toronto's...,SAFE
5,2024,9,3,8.0,Woodbine Beaches,19.0,E.coli levels are within the City of Toronto's...,SAFE
6,2024,9,2,8.0,Woodbine Beaches,20.0,E.coli levels are within the City of Toronto's...,SAFE
7,2024,9,1,8.0,Woodbine Beaches,28.0,E.coli levels are within the City of Toronto's...,SAFE
8,2024,8,31,8.0,Woodbine Beaches,68.0,E.coli levels are within the City of Toronto's...,SAFE
9,2024,8,30,8.0,Woodbine Beaches,33.0,E.coli levels are within the City of Toronto's...,SAFE


- traditional time series modelling vs NN vs state space models 
- first, statistical correlation between climate and beach readings?
- decide what method of weather forecasting: rolling vs long term 
- circumvent the data sparsity -> deep learning or state space models?